In [17]:
import urllib
from urllib.request import urlopen
from bs4 import BeautifulSoup
import requests
import re
from transformers import BartTokenizer, BartForConditionalGeneration
from transformers import T5Tokenizer, T5ForConditionalGeneration, T5Config
import torch
import os, sys
import subprocess
from pptx import Presentation
from math import floor
import time
import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
import cv2
from pptx.util import Inches
import pickle

def geturl(var):
    url_source = urlopen(var).read()
    soup = BeautifulSoup(url_source,'lxml')

    var1= soup.find_all("a",class_="image")
    i=""
    for var in var1:
      i=i+str(var)

    img=[]

    for j in range(len(i)):
      if(j<(len(i)-3)):
        if(i[j]=='c' and i[j+1]=='=' and i[j+2]=='"'):
          h=""
          for k in range(j+3,len(i)):
            if(i[k]=='"'):
                break
            h=h+i[k]
          img.append(h)
    des=[]
    mydivs = soup.find_all("div", {"class": "thumbinner"})
    for i in mydivs:
      des.append(i.text)
    return img,des

def getnarray(var):

    with urllib.request.urlopen(var) as url:
        s = url.read()

    arr = np.asarray(bytearray(s), dtype=np.uint8)
    img = cv2.imdecode(arr, -1) 

    return img


def joining(img,text):
    cv2.imwrite('i1.png', img)
    n = len(text)
    print(img.shape)
    (width, height, dim) = img.shape
    flag = height // 10 
    s = text.split()
    down = 0
    count = 0
    li = []
    temp = ''
    for i in s:
        count += len(i)
        if count > flag:
            count = len(i)
            down += 1
            li.append(temp)
            temp = ''
        temp = temp + i + ' '
        count += 1
    down += 1
    li.append(temp)
    print(li)
    down = n // flag + 1
    print(width, height)
    temp = np.ones((10 + down * 20, height, 3)) * 255
    print(temp.shape)
    
    if dim == 4:
        b,g,r, a = cv2.split(img)

        new_img  = cv2.merge((b, g, r))
        not_a = cv2.bitwise_not(a)
        not_a = cv2.cvtColor(not_a, cv2.COLOR_GRAY2BGR)

        new_img = cv2.bitwise_and(new_img,new_img,mask = a)
        img = cv2.add(new_img, not_a)

    img1 = np.concatenate((img, temp), axis = 0)
    cv2.imwrite('i2.jpg', img1)
    print(img1.shape)
    font = cv2.FONT_HERSHEY_SIMPLEX
    org = (0, width + 20)
    fontScale = 0.6
    color = (0, 0, 0)
    thickness = 2
    count = 1
    for i in li:
        org = (0, width + count * 20)
        image = cv2.putText(img1, i, org, font, 
                       fontScale, color, thickness, cv2.LINE_AA)
        count += 1

    cv2.imwrite('i.png', image)
    return(image)

print("Input:-")
p = input()
INPUT = p
t=""
for i in range(len(p)):
  if(p[i]==' '):
    t=t+"_"
  else:
    t=t+p[i]

var='https://en.wikipedia.org/wiki/'+t
# Specifying the url of the web page
url_source = urlopen(var).read()

imlist,urlist = geturl(var)
count = 0
imlist1 = []
urlist1 = []
print(len(imlist), len(urlist))
for i in imlist:
  if 'gif' not in i:
    imlist1.append(imlist[count])
    try:
      urlist1.append(urlist[count])
    except:
      urlist1.append('')
  count += 1
urlist = urlist1[0:8]
imlist = imlist1[0:8]
print(urlist)
num_img = len(imlist)
print(num_img)
# Using beautifulsoup library to get the html file from the url
soup = BeautifulSoup(url_source,'lxml')
soup

# Extracting the text content from the paragraphs by finding the p tag
para=[]
para2=[]
for header in soup.find_all(['h1', 'h2','p']):
  para.append(header.get_text())
for header in soup.find_all('p'):
  para2.append(header.get_text())
l=[]
for i in soup.find_all(['h1','h2']):
  l.append(i.get_text())
print(l)
para.pop(0)


for i in range(len(l)):
  r=l[i]
  value2=""
  for x in range(len(r)):
    if(r[x]=='['):
      break
    else:
      value2=value2+r[x]
  l[i]=value2
l.pop(0)
l.pop(0)
print(l)
# Adding all pragraph in a single sring for further use

final="Introduction"
final2="Introduction"
for val in para:
  final=final+" "+str(val)
final = re.sub(r"\[.*?\]+", '',final)
final = final.replace('\n', '')
print("Output:-")
print(final)
for val in para2:
  final2=final2+" "+str(val)
final2 = re.sub(r"\[.*?\]+", '',final2)
final2 = final2.replace('\n', '')
print(final2)
fun=[]
st=""
j=0
odd=0
print(len(final))
print(len(final2))
for i in range(len(final)):
  if(j>=len(final2)):
    break
  if(final[i]==final2[j]):
    st=st+final[i]
    j=j+1
    odd=0
  else:
    if(odd==0):
      fun.append(st)
      st=""
      st=st+final[i]
      odd=1
    else:
      st=st+final[i]
fun.append(st)

for i in fun:
  print('-'*50)
  print(i)
fun2=[]
last=0
l.insert(0,'Introduction')
fun3=[]
for i in fun:
  z1=i
  y1=len(l[last])
  last=last+1
  y1=y1+1
  if(last==2):
    y1=y1+8
  z2=z1[y1:]
  fun3.append(z2)
last=0
for i in fun3:
  fun2.append(l[last])
  last=last+1
  fun2.append(i)
it=iter(fun2)
result = dict(zip(it, it))

# loading fine-tuned model
with open('model.pickle', 'rb') as handle:
    model = pickle.load(handle)
with open('tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

count = 0
final_output = {}
for i in result:
  ARTICLE_TO_SUMMARIZE = result[i][0:min(len(result[i]), 3500)]

  print(i, len(ARTICLE_TO_SUMMARIZE))

  inputs = tokenizer([ARTICLE_TO_SUMMARIZE], max_length=len(ARTICLE_TO_SUMMARIZE), return_tensors="pt")

  # Generate Summary
  summary_ids = model.generate(inputs["input_ids"], num_beams=4, max_length=100)
  ans = tokenizer.batch_decode(summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)
  final_output[i] = ans[0]
  count += 1
  if count > 6:
    break

length = 8
copy1 = final_output
design = int(input("Choose a template "))

if(design == 1):
    prs = Presentation(r"FYPtemplates\1.pptx")
elif(design == 2):
    prs = Presentation(r"FYPtemplates\2.pptx")
elif(design == 3):
    prs = Presentation(r"FYPtemplates\3.pptx")
elif(design == 4):
    prs = Presentation(r"FYPtemplates\4.pptx")
elif(design == 5):
    prs = Presentation(r"FYPtemplates\5.pptx")
elif(design == 6):
    prs = Presentation(r"FYPtemplates\6.pptx")
elif(design == 7):
    prs = Presentation(r"FYPtemplates\7.pptx")
elif(design == 8):
    prs = Presentation(r"FYPtemplates\8.pptx")

num = 3

#Title Page Design
slide_layout = prs.slide_layouts[0]
slide1 = prs.slides.add_slide(slide_layout)
title = slide1.shapes.title
title.text = p

titles_for_slide = []
for x in copy1:
    titles_for_slide.append(x)

nch = min(8, num_img, len(titles_for_slide))    
for i in range(nch-1):
    imarray = getnarray("http:"+imlist[i])
    im = Image.fromarray(imarray)
    newsize = (700,700)
    im = im.resize(newsize)

    im.save(r"im_file.png")

    slide = prs.slides.add_slide(prs.slide_layouts[3])
    shapes = slide.shapes
    
    title_shape = shapes.title
    body_shape = shapes.placeholders[1]
    
    title_shape.text = titles_for_slide[i]
    
    img_path = r'im_file.png'

    left = Inches(8)
    top = Inches(2)
    height = Inches(3)
    width = Inches(4)
    pic = slide.shapes.add_picture(img_path, left, top, height=height)
    
    left = Inches(8)
    top = Inches(5)
    width = Inches(4)
    height = Inches(2)
    txBox = slide.shapes.add_textbox(left, top, width, height)
    tf = txBox.text_frame
    tf.word_wrap = True
    
    p = tf.add_paragraph()
    p.text = urlist[i]
    
    collect = copy1[titles_for_slide[i]].split('.')
    collect = list(filter(None, collect))
    count_of_sentences = len(collect)
    count = 0
        
    text = urlist[i]
    
    for y in range(num):
        if(count == count_of_sentences):
            flag = 0
            break
        else:
            print("here")
            tf = body_shape.text_frame
            p = tf.add_paragraph()
            p.text = collect[count]

            print(collect[count])
            count = count + 1

        
slide = prs.slides.add_slide(prs.slide_layouts[3])
shapes = slide.shapes
title_shape = shapes.title
body_shape = shapes.placeholders[1]
title_shape.text = "Sources"
tf = body_shape.text_frame
p = tf.add_paragraph()
p.text = str(var)
xml_slides = prs.slides._sldIdLst  
slides = list(xml_slides)
xml_slides.remove(slides[0])             
prs.save(r"majid5.pptx")
print(r"PPT Saved")

Input:-
Addition
18 18
[' 3 + 2 = 5 with apples, a popular choice in textbooks[1]', ' The plus sign', ' Columnar addition – the numbers in the column are to be added, with the sum written below the underlined number.', ' Redrawn illustration from The Art of Nombryng, one of the first English arithmetic texts, in the 15th century.[12]', ' One set has 3 polygons while the other set has 2. The total amount of polygons are 5, which is a consequence of the addition of the objects from the two sets (3 + 2 = 5).', ' A number-line visualization of the algebraic addition 2 + 4 = 6. A translation by 2 followed by a translation by 4 is the same as a translation by 6.', ' A number-line visualization of the unary addition 2 + 4 = 6. A translation by 4 is equivalent to four translations by 1.', ' 4 + 2 = 2 + 4 with blocks']
8
['Addition', 'Contents', 'Notation and terminology[edit]', 'Interpretations[edit]', 'Properties[edit]', 'Performing addition[edit]', 'Addition of numbers[edit]', 'Generalizatio

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Introduction 1946
Notation and terminology 2126
Interpretations 1588
Properties 2920
Performing addition 3500
Addition of numbers 3500
Generalizations 2719
Choose a template 2
here
The addition of two whole numbers results in the total amount or sum of those values combined
here
 Addition belongs to arithmetic, a branch of mathematics
here
 Performing addition is one of the simplest numerical tasks to do
here
Addition is written using the plus sign "+" between the terms; that is, in infix notation
here
 The result is expressed with an equals sign
here
 There are also situations where addition is "understood", even though no symbol appears
here
Addition is used to model many physical processes
here
 Even for the simple case of adding natural numbers, there are many possible interpretations and even more visual representations
here
 Possibly the most fundamental interpretation of addition lies in combining sets
here
Addition is commutative, meaning that one can change the order of the te